In [1]:
### instructions:
# 1.
# * copy-paste ipythonwidget.py

# 2. for debug:
# * comment self._init_static()

# 3. for release check:
# * uncomment self._init_static()
# * replace
#    return os.path.join(os.path.dirname(__file__), file_name)
#   with
#    return os.path.join(os.path.dirname('__your_local_repo_dir__/catboost/python-package/catboost/widget/'), file_name)

import os
import csv
import time
import json
from threading import Thread
from IPython.core.display import display, HTML
from traitlets import Unicode, Dict, default
from ipywidgets import DOMWidget, Layout, widget_serialization


class CatboostIpythonWidget(DOMWidget):
    _view_name = Unicode('CatboostIpythonWidgetView').tag(sync=True)
    _view_module = Unicode('catboost_module').tag(sync=True)

    data = Dict({}).tag(sync=True, **widget_serialization)

    def __init__(self, train_dir):
        super(self.__class__, self).__init__()
        self._train_dir = train_dir

    @default('layout')
    def _default_layout(self):
        return Layout(height='500px', align_self='stretch')

    def update_widget(self, subdirs=False):
        # wait for start train (meta.tsv)
        # self._init_static()
        time.sleep(1.0)
        self._update_data(subdirs=subdirs)

        display(self)

        while self._need_update:
            self._update_data(subdirs=subdirs)
            time.sleep(2.0)

    def _run_update(self):
        thread = Thread(target=self.update_widget, args=())
        thread.start()

    def _get_subdirectories(self, a_dir):
        return [{'name': name, 'path': os.path.join(a_dir, name)}
            for name in os.listdir(a_dir) if os.path.isdir(os.path.join(a_dir, name))]

    def _update_data(self, subdirs=False):
        data = {}
        dirs = [{'name': 'current', 'path': self._train_dir}]
        need_update = False

        if subdirs:
            dirs = self._get_subdirectories(self._train_dir)

        for dir_info in dirs:
            path = dir_info.get('path')
            content = self._update_data_from_dir(path)

            if not content:
                continue

            data[path] = {
                'path': path,
                'name': dir_info.get('name'),
                'content': content
            }

            if not need_update:
                need_update = data[path]['content']['passed_iterations'] + 1 < data[path]['content']['total_iterations']

        self.data = data
        self._need_update = need_update

    def _update_data_from_dir(self, path):
        data = {
            'iterations': {},
            'meta': {}
        }
        
        training_json = os.path.join(path, 'catboost_training.json')
        
        if os.path.isfile(training_json):
             with open(training_json, 'r') as json_data:
                training_data = json.load(json_data)
                
                data['meta'] = training_data['meta']
                data['iterations'] = training_data['iterations']
                   
        else:
            return None

        return {
            'passed_iterations': data['iterations'][-1]['iteration'],
            'total_iterations': data['meta']['iteration_count'],
            'data': data
        }

    @staticmethod
    def _get_static_path(file_name):
        return os.path.join(os.path.dirname('/Users/ivan-karev/github/catboost/catboost/python-package/catboost/widget/'), file_name)

    def _init_static(self):
        with open(self._get_static_path('CatboostIpython.css')) as f:
            css = f.read()
        js = ''

        # never use require in your projects
        js += 'window.__define = window.define;window.__require = window.require;window.define = undefined;window.require = undefined;'
        with open(self._get_static_path('plotly-basic.min.js')) as f:
            js += f.read()
        js += 'window.define = window.__define;window.require = window.__require;window.__define = undefined; window.__require = undefined;'

        with open(self._get_static_path('CatboostIpythonPlotly.js')) as f:
            js += f.read()
        with open(self._get_static_path('CatboostIpythonInit.js')) as f:
            js += f.read()
        html = """
            <style>
                {}
            </style>
            <script>
                {}
            </script>
        """.format(css, js)

        display(HTML(html))

In [9]:
%%javascript

// http://stackoverflow.com/questions/32170197/how-do-can-i-use-a-custom-js-file-under-jupyter-notebook

// instructions:
// 1.
// copy-paste CatboostIpythonInit.js

// 2.
// replace
//  var debug = false;
// with
//  var debug = true;

// 3. setup local notebook:
/*
mkdir ~/.jupyter/custom
cd ~/.jupyter/custom
ln -s __your_local_repo_dir__/catboost/python-package/catboost/widget/CatboostIpythonPlotly.js CatboostIpythonPlotly.js
ln -s __your_local_repo_dir__/catboost/python-package/catboost/widget/CatboostIpython.css CatboostIpython.css
ln -s __your_local_repo_dir__/catboost/python-package/catboost/widget/plotly-basic.min.js plotly-basic.min.js
*/


var debug = true;

if (debug) {
    require.config({
        shim:{
            "custom/CatboostIpythonPlotly":{
                deps:["custom/plotly-basic.min"]
            } 
        }
    })
    
    require.undef('catboost_module');
    require.undef('custom/CatboostIpythonPlotly');
}

var moduleBase = '@jupyter-widgets/base';
var modules = [moduleBase];

if (debug) {
    modules.push('custom/CatboostIpythonPlotly');
}

define('catboost_module', modules, function(widgets) {
    var getInstance = function(el) {
            var id = $(el).attr('catboost-id');

            if (!id) {
                return null;
            }

            id = id.replace('catboost_', '');

            if (!window.catboostIpythonInstances[id]) {
                return null;
            }

            return window.catboostIpythonInstances[id];
        },
        addInstance = function(el) {
            $(el).attr('catboost-id', 'catboost_' + window.catboostIpythonIndex);

            var catboostIpython = new CatboostIpython();
            catboostIpython.index = window.catboostIpythonIndex;
            catboostIpython.plotly = window.Plotly;
            if (debug) {
                catboostIpython.loadStyles('/custom/CatboostIpython.css', function(){catboostIpython.resizeCharts();})
            }

            window.catboostIpythonInstances[window.catboostIpythonIndex] = catboostIpython;

            window.catboostIpythonIndex++;

            return catboostIpython;
        };

    var CatboostIpythonWidget = widgets.DOMWidgetView.extend({

        initialize: function() {
            CatboostIpythonWidget.__super__.initialize.apply(this, arguments);

            if (!window.catboostIpythonInstances) {
                window.catboostIpythonInstances = {};
            }

            if (typeof window.catboostIpythonIndex === 'undefined') {
                window.catboostIpythonIndex = 0;
            }

            var catboostIpythonInstance = getInstance(this.el);

            if (!catboostIpythonInstance) {
                catboostIpythonInstance = addInstance(this.el);
            }

            catboostIpythonInstance.init();
        },

        render: function() {
            this.value_changed();
            this.model.on('change:value', this.value_changed, this);
        },

        update: function() {
            this.value_changed();
        },

        value_changed: function() {
            this.el.style['width'] = this.model.get('width');
            this.el.style['height'] = this.model.get('height');
            this.displayed.then(_.bind(this.render_charts, this));
        },

        process_all: function(parent, params) {
            var data = params.data;
            
            //console.log('data', data);

            for (var path in data) {
                if (data.hasOwnProperty(path)) {
                    this.process_row(parent, data[path])
                }
            }
        },

        process_row: function(parent, data) {
            var catboostIpython = getInstance(parent),
                path = data.path,
                content = data.content,
                items = content.data.iterations,
                firstIndex = 0,
                chunks = [];
            
            if (!items || !items.length) {
                return;
            }
            
            if (!catboostIpython.lastIndex) {
                catboostIpython.lastIndex = {} 
            }

            if (catboostIpython.lastIndex[path]) {
                firstIndex = catboostIpython.lastIndex[path] + 1;
            }

            catboostIpython.lastIndex[path] = items.length - 1;

            for (var i = firstIndex; i < items.length; i++) {
                chunks.push(items[i]);
            }

            catboostIpython.addMeta(data.path, content.data.meta);

            catboostIpython.addPoints(parent, {
                chunks: chunks,
                train: data.name,
                path: data.path
            });
        },

        render_charts: function () {
            this.process_all(this.el, {
                data: this.model.get('data')
            });

            return this;
        }
    });

    return {
        CatboostIpythonWidgetView: CatboostIpythonWidget
    };
});

<IPython.core.display.Javascript object>

In [4]:
w = CatboostIpythonWidget('new_train')
w.update_widget()

CatboostIpythonWidget(data={'new_train': {'content': {'data': {'meta': {u'name': u'experiment', u'test_metrics': [{u'name': u'Logloss', u'value': u'min'}, {u'name': u'RMSE', u'value': u'min'}, {u'name': u'AUC', u'value': u'max'}], u'learn_sets': [u'learn'], u'learn_metrics': [{u'name': u'Logloss', u'value': u'min'}, {u'name': u'RMSE', u'value': u'min'}, {u'name': u'AUC', u'value': u'max'}], u'iteration_count': 20, u'test_sets': [u'test']}, 'iterations': [{u'test': [0.6600097926, 0.8186408447, 0.7882716049], u'passed_time': 0.01195065927, u'iteration': 0, u'remaining_time': 0.2270625262, u'learn': [0.6565050203, 0.8045194116, 0.9427502877]}, {u'test': [0.632232761, 0.7521424448, 0.8009259259], u'passed_time': 0.01321400713, u'iteration': 1, u'remaining_time': 0.1189260641, u'learn': [0.6255581951, 0.7309581064, 0.9462025316]}, {u'test': [0.6095533415, 0.6986701294, 0.8104938272], u'passed_time': 0.01747301169, u'iteration': 2, u'remaining_time': 0.0990137329, u'learn': [0.5972445019, 0.6674712405, 0.9571346375]}, {u'test': [0.5900602047, 0.6505211024, 0.8703703704], u'passed_time': 0.01988600819, u'iteration': 3, u'remaining_time': 0.07954403276, u'learn': [0.576111189, 0.6163792864, 0.9833141542]}, {u'test': [0.5665918765, 0.5981484265, 0.8734567901], u'passed_time': 0.02221930921, u'iteration': 4, u'remaining_time': 0.06665792764, u'learn': [0.5480418822, 0.551996609, 0.9907940161]}, {u'test': [0.5510878118, 0.5675146714, 0.8666666667], u'passed_time': 0.02381639127, u'iteration': 5, u'remaining_time': 0.05557157962, u'learn': [0.530200324, 0.5112019609, 0.9838895282]}, {u'test': [0.5371883975, 0.5458694973, 0.862345679], u'passed_time': 0.02759446774, u'iteration': 6, u'remaining_time': 0.05124686866, u'learn': [0.5093274822, 0.4693390275, 0.9815880322]}, {u'test': [0.5196093687, 0.5164592834, 0.8641975309], u'passed_time': 0.02926329191, u'iteration': 7, u'remaining_time': 0.04389493786, u'learn': [0.4892220617, 0.4324720121, 0.9867663982]}, {u'test': [0.514718047, 0.4997642919, 0.8641975309], u'passed_time': 0.03006744424, u'iteration': 8, u'remaining_time': 0.03674909852, u'learn': [0.4848357164, 0.4135370112, 0.9867663982]}, {u'test': [0.5070544057, 0.4767807491, 0.8648148148], u'passed_time': 0.03104546668, u'iteration': 9, u'remaining_time': 0.03104546668, u'learn': [0.476894303, 0.391844182, 0.9884925201]}, {u'test': [0.4884107139, 0.4656478835, 0.8691358025], u'passed_time': 0.03491730462, u'iteration': 10, u'remaining_time': 0.02856870378, u'learn': [0.4559441537, 0.368178344, 0.9942462601]}, {u'test': [0.475222362, 0.4649231047, 0.8691358025], u'passed_time': 0.03872574833, u'iteration': 11, u'remaining_time': 0.02581716555, u'learn': [0.4395892343, 0.3601795584, 0.9902186421]}, {u'test': [0.4671390682, 0.4702490774, 0.8635802469], u'passed_time': 0.03969476613, u'iteration': 12, u'remaining_time': 0.02137410484, u'learn': [0.4299335594, 0.3634165349, 0.9856156502]}, {u'test': [0.4593684635, 0.4640512426, 0.8709876543], u'passed_time': 0.04063294628, u'iteration': 13, u'remaining_time': 0.01741411983, u'learn': [0.4239420933, 0.3563966828, 0.9907940161]}, {u'test': [0.4580933529, 0.4878625883, 0.8546296296], u'passed_time': 0.04445887295, u'iteration': 14, u'remaining_time': 0.01481962432, u'learn': [0.4128026493, 0.3622160541, 0.9896432681]}, {u'test': [0.4485422823, 0.4976934801, 0.8614197531], u'passed_time': 0.04693425378, u'iteration': 15, u'remaining_time': 0.01173356344, u'learn': [0.4014032279, 0.3732812248, 0.9907940161]}, {u'test': [0.440040623, 0.52335625, 0.8682098765], u'passed_time': 0.05097190937, u'iteration': 16, u'remaining_time': 0.00899504283, u'learn': [0.3894202204, 0.4013746303, 0.9936708861]}, {u'test': [0.4323807262, 0.5508922729, 0.862037037], u'passed_time': 0.05450196203, u'iteration': 17, u'remaining_time': 0.006055773559, u'learn': [0.3762092106, 0.4249888747, 0.9919447641]}, {u'test': [0.426786483, 0.5779252281, 0.8601851852], u'passed_time': 0.05616939042, u'iteration': 18, u'remaini

In [5]:
w = CatboostIpythonWidget('new_train_gaps')
w.update_widget()

CatboostIpythonWidget(data={'new_train_gaps': {'content': {'data': {'meta': {u'name': u'experiment', u'test_metrics': [{u'name': u'Logloss', u'value': u'min'}, {u'name': u'RMSE', u'value': u'min'}, {u'name': u'AUC', u'value': u'max'}], u'learn_sets': [u'learn'], u'learn_metrics': [{u'name': u'Logloss', u'value': u'min'}, {u'name': u'RMSE', u'value': u'min'}, {u'name': u'AUC', u'value': u'max'}], u'iteration_count': 20, u'test_sets': [u'test']}, 'iterations': [{u'test': [0.6600097926, 0.8186408447, 0.7882716049], u'passed_time': 0.01195065927, u'iteration': 0, u'remaining_time': 0.2270625262, u'learn': [0.6565050203, 0.8045194116, 0.9427502877]}, {u'test': [0.6095533415, 0.6986701294, 0.8104938272], u'passed_time': 0.01747301169, u'iteration': 2, u'remaining_time': 0.0990137329, u'learn': [0.5972445019, 0.6674712405, 0.9571346375]}, {u'test': [0.5665918765, 0.5981484265, 0.8734567901], u'passed_time': 0.02221930921, u'iteration': 4, u'remaining_time': 0.06665792764, u'learn': [0.5480418822, 0.551996609, 0.9907940161]}, {u'test': [0.5371883975, 0.5458694973, 0.862345679], u'passed_time': 0.02759446774, u'iteration': 6, u'remaining_time': 0.05124686866, u'learn': [0.5093274822, 0.4693390275, 0.9815880322]}, {u'test': [0.5070544057, 0.4767807491, 0.8648148148], u'passed_time': 0.03104546668, u'iteration': 9, u'remaining_time': 0.03104546668, u'learn': [0.476894303, 0.391844182, 0.9884925201]}, {u'test': [0.4884107139, 0.4656478835, 0.8691358025], u'passed_time': 0.03491730462, u'iteration': 10, u'remaining_time': 0.02856870378, u'learn': [0.4559441537, 0.368178344, 0.9942462601]}, {u'test': [0.4580933529, 0.4878625883, 0.8546296296], u'passed_time': 0.04445887295, u'iteration': 14, u'remaining_time': 0.01481962432, u'learn': [0.4128026493, 0.3622160541, 0.9896432681]}, {u'test': [0.4224230192, 0.6078599851, 0.8589506173], u'passed_time': 0.05740768101, u'iteration': 19, u'remaining_time': 0, u'learn': [0.359672552, 0.4794579543, 0.9919447641]}]}, 'total_iterations': 20, 'passed_iterations': 19}, 'path': 'new_train_gaps', 'name': 'current'}}, layout=Layout(align_self=u'stretch', height=u'500px'))

In [ ]:
w = CatboostIpythonWidget('new_train_with_nan')
w.update_widget()

CatboostIpythonWidget(data={'new_train_with_nan': {'content': {'data': {'meta': {u'name': u'experiment', u'test_metrics': [{u'name': u'MAPE', u'value': u'min'}, {u'name': u'RMSE', u'value': u'min'}, {u'name': u'AUC', u'value': u'max'}], u'learn_sets': [u'learn'], u'learn_metrics': [{u'name': u'MAPE', u'value': u'min'}, {u'name': u'RMSE', u'value': u'min'}, {u'name': u'AUC', u'value': u'max'}], u'iteration_count': 20, u'test_sets': [u'test']}, 'iterations': [{u'test': [u'inf', u'inf', 0.5], u'passed_time': 0.0009625287001, u'iteration': 0, u'remaining_time': 0.0182880453, u'learn': [u'inf', u'inf', 0.5]}, {u'test': [u'nan', u'nan', 1], u'passed_time': 0.001608976648, u'iteration': 1, u'remaining_time': 0.01448078983, u'learn': [u'nan', u'nan', 1]}]}, 'total_iterations': 20, 'passed_iterations': 1}, 'path': 'new_train_with_nan', 'name': 'current'}}, layout=Layout(align_self=u'stretch', height=u'500px'))

In [7]:
w = CatboostIpythonWidget('new_train_period')
w.update_widget()

CatboostIpythonWidget(data={'new_train_period': {'content': {'data': {'meta': {u'name': u'experiment', u'test_metrics': [{u'name': u'Logloss', u'value': u'min'}, {u'name': u'RMSE', u'value': u'min'}, {u'name': u'AUC', u'value': u'max'}], u'learn_sets': [u'learn'], u'learn_metrics': [{u'name': u'Logloss', u'value': u'min'}, {u'name': u'RMSE', u'value': u'min'}, {u'name': u'AUC', u'value': u'max'}], u'iteration_count': 20, u'test_sets': [u'test']}, 'iterations': [{u'test': [0.6600097926, 0.8186408447, 0.7882716049], u'passed_time': 0.01195065927, u'iteration': 0, u'remaining_time': 0.2270625262, u'learn': [0.6565050203, 0.8045194116, 0.9427502877]}, {u'test': [0.5510878118, 0.5675146714, 0.8666666667], u'passed_time': 0.02381639127, u'iteration': 5, u'remaining_time': 0.05557157962, u'learn': [0.530200324, 0.5112019609, 0.9838895282]}, {u'test': [0.4884107139, 0.4656478835, 0.8691358025], u'passed_time': 0.03491730462, u'iteration': 10, u'remaining_time': 0.02856870378, u'learn': [0.4559441537, 0.368178344, 0.9942462601]}, {u'test': [0.4485422823, 0.4976934801, 0.8614197531], u'passed_time': 0.04693425378, u'iteration': 19, u'remaining_time': 0.01173356344, u'learn': [0.4014032279, 0.3732812248, 0.9907940161]}]}, 'total_iterations': 20, 'passed_iterations': 19}, 'path': 'new_train_period', 'name': 'current'}}, layout=Layout(align_self=u'stretch', height=u'500px'))

In [11]:
w = CatboostIpythonWidget('new_train_no_time')
w.update_widget()

CatboostIpythonWidget(data={'new_train_no_time': {'content': {'data': {'meta': {u'name': u'experiment', u'test_metrics': [], u'learn_sets': [u'learn'], u'learn_metrics': [{u'name': u'Logloss', u'value': u'min'}, {u'name': u'RMSE', u'value': u'min'}, {u'name': u'AUC', u'value': u'max'}], u'iteration_count': 20, u'test_sets': []}, 'iterations': [{u'iteration': 0, u'learn': [0.6565050203, 0.8045194116, 0.9427502877]}, {u'iteration': 1, u'learn': [0.6255581951, 0.7309581064, 0.9462025316]}, {u'iteration': 2, u'learn': [0.5972445019, 0.6674712405, 0.9571346375]}, {u'iteration': 3, u'learn': [0.5691014602, 0.6047088974, 0.9833141542]}, {u'iteration': 4, u'learn': [0.5416648099, 0.5427365234, 0.9907940161]}, {u'iteration': 5, u'learn': [0.5242379629, 0.5037694846, 0.9821634062]}, {u'iteration': 6, u'learn': [0.5037993188, 0.4641605473, 0.9792865362]}, {u'iteration': 7, u'learn': [0.4841199805, 0.4299345297, 0.9838895282]}, {u'iteration': 8, u'learn': [0.4798038573, 0.4115503627, 0.9838895282]}, {u'iteration': 9, u'learn': [0.4719679875, 0.3907662598, 0.9844649022]}, {u'iteration': 10, u'learn': [0.4513898411, 0.3705063459, 0.9919447641]}, {u'iteration': 11, u'learn': [0.435328002, 0.3653144172, 0.9902186421]}, {u'iteration': 12, u'learn': [0.425952353, 0.3707377884, 0.9879171461]}, {u'iteration': 13, u'learn': [0.4200576972, 0.3648080537, 0.9907940161]}, {u'iteration': 14, u'learn': [0.4091305988, 0.3721025333, 0.9890678941]}, {u'iteration': 15, u'learn': [0.397930343, 0.3845998151, 0.9879171461]}, {u'iteration': 16, u'learn': [0.3861739072, 0.4137267368, 0.9907940161]}, {u'iteration': 17, u'learn': [0.3731614753, 0.4381811459, 0.9919447641]}, {u'iteration': 18, u'learn': [0.3643969591, 0.4647138998, 0.9913693901]}, {u'iteration': 19, u'learn': [0.3569188272, 0.4931260535, 0.9913693901]}]}, 'total_iterations': 20, 'passed_iterations': 19}, 'path': 'new_train_no_time', 'name': 'current'}}, layout=Layout(align_self=u'stretch', height=u'500px'))